In [1]:
import pandas as pd
import skimage
import numpy as np
import os

/tmp/ipykernel_114222/1646939490.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
data = pd.read_csv('/home/jaalzate/Prostate_Cancer_TFM/Files/Bounding_Boxes/Only_Bounding.csv')

## Save Bounding Boxes to Nifti

In [39]:
import dicom2nifti
import pydicom
import SimpleITK as sitk

In [40]:
def _flatten(t):
    return [item for sublist in t for item in sublist]


def crop(image, margin, interpolator=sitk.sitkLinear):
    """
    Crops a sitk.Image while retaining correct spacing. Negative margins will lead to zero padding

    Args:
        image:  a sitk.Image
        margin: margins to crop. Single integer or float (percentage crop),
                lists of int/float or nestes lists are supported.
    """
    if isinstance(margin, (list, tuple)):
        assert len(margin) == 3, "expected margin to be of length 3"
    else:
        assert isinstance(margin, (int, float)), "expected margin to be a float value"
        margin = [margin, margin, margin]

    margin = [m if isinstance(m, (tuple, list)) else [m, m] for m in margin]
    old_size = image.GetSize()

    # calculate new origin and new image size
    if all([isinstance(m, float) for m in _flatten(margin)]):
        assert all([m >= 0 and m < 0.5 for m in _flatten(margin)]), "margins must be between 0 and 0.5"
        to_crop = [[int(sz * _m) for _m in m] for sz, m in zip(old_size, margin)]
    elif all([isinstance(m, int) for m in _flatten(margin)]):
        to_crop = margin
    else:
        raise ValueError("Wrong format of margins.")

    new_size = [sz - sum(c) for sz, c in zip(old_size, to_crop)]

    # origin has Index (0,0,0)
    # new origin has Index (to_crop[0][0], to_crop[2][0], to_crop[2][0])
    new_origin = image.TransformIndexToPhysicalPoint([c[0] for c in to_crop])

    # create reference plane to resample image
    ref_image = sitk.Image(new_size, image.GetPixelIDValue())
    ref_image.SetSpacing(image.GetSpacing())
    ref_image.SetOrigin(new_origin)
    ref_image.SetDirection(image.GetDirection())

    return sitk.Resample(image, ref_image, interpolator=interpolator)

In [41]:
path ="/nvmescratch/ceib/Prostate/Bounding_Boxes"

In [48]:
data

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality,Path_Segmentation
0,5586,6040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
1,5586,6040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
2,5596,6050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
3,5700,6173,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/SEG_20240113_211959_757...,AXIAL DW 1500 PROSTATA,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
4,5700,6173,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/SEG_20240113_211927_274...,AXIAL DW 1000 PROSTATA,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
...,...,...,...,...,...,...,...,...,...,...,...
530,5646,6108,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/SEG_20240113_210039_451...,AXI PELVIS DWI B50-800,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
531,5646,6108,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/SEG_20240113_205921_383...,ADC (10^-6 mm²/s):Nov 26 2019 16-59-11 CET,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,ADC,NaN
532,5646,6108,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/SEG_20240113_205854_334...,DW_Synthetic: Ax FOCUS B-50-800 Synthetic B-2000,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
533,5399,5819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...


In [89]:
bounding_data_t2 = data[data.modality=="T2"].reset_index(drop=True)

In [91]:
bounding_data_t2

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality,Path_Segmentation
0,5586,6040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
1,5586,6040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
2,5596,6050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
3,5700,6173,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/SEG_20240113_211858_507...,AXIAL T2 ALTA RES. (de abajo arriba en próstata),/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
4,5665,6131,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/SEG_20240115_203626_074...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
...,...,...,...,...,...,...,...,...,...,...,...
292,5636,6097,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/SEG_20240113_202733_764...,T2W_TSE_TRA_IQ,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
293,5613,6070,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/SEG_20240113_203814_089...,AX T2 PROPELLER,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
294,5646,6108,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/SEG_20240113_210319_135...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
295,5399,5819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...


In [90]:
os.path.dirname(os.path.join(path,'Cropped',*bounding_data_t2.image_nifti[0].split('/')[-5:]))

'/nvmescratch/ceib/Prostate/Bounding_Boxes/Cropped/005586/006040/8/NIFTI'

In [95]:
bounding_data_t2['cropped_image'] = None
bounding_data_t2['cropped_bounding_box'] = None

for i,row in bounding_data_t2.iterrows():

    bounding_imgs = os.listdir(os.path.join(path,row.bounding_box))
    bounding_imgs = [img for img in bounding_imgs if img.endswith('.dcm')]

    org_img = sitk.ReadImage(row['image_nifti'])
    img_crop = crop(org_img, margin=[0.2,0.2,0.0])

    if len(bounding_imgs) > 1:
        print('Image {} has more than one bounding box'.format(row['image_nifti']))

    for img in bounding_imgs:
        img_path = os.path.join(path,row.bounding_box,img)
        b_img = sitk.ReadImage(img_path)
        b_img_crop = crop(b_img, margin=[0.2,0.2,0.0],interpolator=sitk.sitkNearestNeighbor)
        b_img_resampled = sitk.Resample(b_img_crop, img_crop, interpolator=sitk.sitkNearestNeighbor)
        b_new_path = os.path.join(path,'Cropped',row.bounding_box,img.replace('.dcm','.nii.gz'))
        bounding_data_t2.at[i,'cropped_bounding_box'] = b_new_path
        #print('Image {} has been cropped'.format(img_path))

        os.makedirs(os.path.dirname(b_new_path), exist_ok=True)

        sitk.WriteImage(b_img_resampled, b_new_path)
    
    img_new_path = os.path.join(path,'Cropped',*row.image_nifti.split('/')[-5:])
    bounding_data_t2.at[i,'cropped_image'] = img_new_path
    os.makedirs(os.path.dirname(img_new_path), exist_ok=True)
    sitk.WriteImage(img_crop, img_new_path)

In [96]:
bounding_data_t2

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality,Path_Segmentation,cropped_image,cropped_bounding_box
0,5586,6040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...
1,5586,6040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...
2,5596,6050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...
3,5700,6173,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/SEG_20240113_211858_507...,AXIAL T2 ALTA RES. (de abajo arriba en próstata),/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...
4,5665,6131,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/SEG_20240115_203626_074...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,5636,6097,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/SEG_20240113_202733_764...,T2W_TSE_TRA_IQ,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...
293,5613,6070,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/SEG_20240113_203814_089...,AX T2 PROPELLER,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...
294,5646,6108,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/SEG_20240113_210319_135...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...
295,5399,5819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...,/nvmescratch/ceib/Prostate/Bounding_Boxes/Crop...


## Crop Prostate

In [5]:
images_t2 = data[data['modality'] == 'T2'].reset_index(drop=True)

In [ ]:
def crop_image(image, bx, by):
    return image[by[0] : by[2], bx[0] : bx[1]]


def calculate_gland(
    pre_img_1,
    nii_img,
    nii_label,
    nii_gland,
    img_transform,
    plot=False,
    label_exists=True,
):
    prediction_made = False
    pre_img_1 = monai.transforms.ToTensor()(np.expand_dims(monai.transforms.DivisiblePad(16)(pre_img), axis=0))
    with torch.no_grad():
        test_volume = pre_img_1.to(device)
        test_outputs = model(test_volume)
        test_outputs = test_outputs > 0.5
        areas_max = []

        seg = np.array(test_outputs.detach().cpu()[0, 1, :, :, :])
        img = np.array(pre_img_1[0, 0, :, :, :])
        ########################################################################
        # Segmentation
        for i in range(img.shape[-1]):
            labels = measure.label(seg[:, :, i])
            regions = regionprops(labels)
            props = measure.regionprops(labels, img[:, :, i])
            areas = [getattr(prop, "area") for prop in props]

            if len(areas) > 0:
                props = regions[np.argmax(areas)]
                minr, minc, maxr, maxc = props.bbox
                bx = (minc - 60, maxc + 60, maxc + 60, minc - 60, minc - 60)
                by = (minr - 20, minr - 20, maxr + 20, maxr + 20, minr - 20)
                areas_max.append(areas[np.argmax(areas)])
            else:
                areas_max.append(0)

        seg = monai.transforms.Resize(spatial_size=(nii_img.shape[0], nii_img.shape[1], -1))(
            monai.transforms.EnsureChannelFirst(strict_check=False, channel_dim="no_channel")(seg)
        )[0]
        img = monai.transforms.Resize(spatial_size=(nii_img.shape[0], nii_img.shape[1], -1))(
            monai.transforms.EnsureChannelFirst(strict_check=False, channel_dim="no_channel")(img)
        )[0]

        labels = measure.label(seg[:, :, np.argmax(areas_max)])
        regions = regionprops(labels)
        props = measure.regionprops(labels, img[:, :, np.argmax(areas_max)])
        areas = [getattr(prop, "area") for prop in props]
        if len(areas) > 0:
            prediction_made = True
            props = regions[np.argmax(areas)]
            minr, minc, maxr, maxc = props.bbox
            bx = (minc - 60, maxc + 60, maxc + 60, minc - 60, minc - 60)
            by = (minr - 20, minr - 20, maxr + 20, maxr + 20, minr - 20)
            bb1 = {"x1": bx[0], "x2": bx[1], "y1": by[1], "y2": by[2]}
        else:
            bb1 = {"x1": 0, "x2": 1, "y1": 0, "y2": 1}
        ###################################################################
        # Original image
        # areas_max = []
        # for i in range(nii_img.shape[-1]):
        #     labels = measure.label(nii_gland[:, :, i])
        #     regions = regionprops(labels)
        #     props = measure.regionprops(labels, nii_img[:, :, i])
        #     areas = [getattr(prop, "area") for prop in props]

        #     if len(areas) > 0:
        #         props = regions[np.argmax(areas)]
        #         minrg, mincg, maxrg, maxcg = props.bbox
        #         bxg = (mincg - 60, maxcg + 60, maxcg + 60, mincg - 60, mincg - 60)
        #         byg = (minrg - 20, minrg - 20, maxrg + 20, maxrg + 20, minrg - 20)
        #         areas_max.append(areas[np.argmax(areas)])
        #     else:
        #         areas_max.append(0)

        # labels = measure.label(nii_gland[:, :, np.argmax(areas_max)])
        # regions = regionprops(labels)
        # props = measure.regionprops(labels, nii_img[:, :, np.argmax(areas_max)])
        # areas = [getattr(prop, "area") for prop in props]
        # props = regions[np.argmax(areas)]
        # minrg, mincg, maxrg, maxcg = props.bbox
        # bxg = (mincg - 60, maxcg + 60, maxcg + 60, mincg - 60, mincg - 60)
        # byg = (minrg - 20, minrg - 20, maxrg + 20, maxrg + 20, minrg - 20)
        # bb2 = {"x1": bxg[0], "x2": bxg[1], "y1": byg[1], "y2": byg[2]}
        # img_plot=Resize(spatial_size=(384,384,-1))(EnsureChannelFirst()(nii_img))[0,:,:,:]
        # label_plot=Resize(spatial_size=(384,384,-1))(EnsureChannelFirst()(nii_label))[0,:,:,:]

        img_plot = monai.transforms.Resize(
            spatial_size=(nii_img.shape[0], nii_img.shape[1], nii_img.shape[2])
        )(
            monai.transforms.EnsureChannelFirst(strict_check=False, channel_dim="no_channel")(
                img_transform
            )
        )[
            0, :, :, :
        ]
        # print(img_plot.shape)

        if label_exists:
            label_plot = nii_label
            # print(label_plot.shape)

        # Get intersection over the union of the bounding boxes
        #iou = get_iou(bb1, bb2)

        # if iou > 0:
        #     bx_u, by_u = get_union(bb1, bb2)
        # else:
        #     bx_u, by_u = bxg, byg
            
        bx_u, by_u = bx, by

        image = []
        label = []

        if label_exists:
            ret_ori = monai.visualize.utils.blend_images(
                np.expand_dims(img_plot, axis=0),
                np.expand_dims(label_plot, axis=0),
                alpha=0.5,
                cmap="hsv",
                rescale_arrays=True,
                transparent_background=True,
            )
        
        for i in range(img_plot.shape[-1]):
            cropped_image = crop_image(img_plot[:, :, i], bx_u, by_u)
            if label_exists:
                cropped_label = crop_image(label_plot[:, :, i], bx_u, by_u)

            if plot:
                plt.figure("check", (18, 6))
                plt.subplot(1, 2, 1)
                plt.title(f"Original image {i+1}")

                if label_exists:
                    plt.imshow(np.transpose(ret_ori[:, :, :, i], axes=[1, 2, 0]))
                else:
                    plt.imshow(img_plot[:, :, i], cmap="gray")

                if prediction_made:
                    plt.plot(bx, by, "-b", linewidth=2.5)
                #plt.plot(bxg, byg, "-r", linewidth=2.5)
                plt.plot(bx_u, by_u, "--y", linewidth=2.5)
                plt.subplot(1, 2, 2)
                plt.title(f"Cropped image {i+1}")
                if label_exists:
                    ret_crop = monai.visualize.utils.blend_images(
                        np.expand_dims(cropped_image, axis=0),
                        np.expand_dims(cropped_label, axis=0),
                        alpha=0.5,
                        cmap="hsv",
                        rescale_arrays=True,
                        transparent_background=True,
                    )
                    plt.imshow(np.transpose(ret_crop, axes=[1, 2, 0]))
                else:
                    plt.imshow(cropped_image, cmap="gray")
                plt.show()
            image.append(cropped_image.cpu().detach().numpy())
            if label_exists:
                label.append(cropped_label)
    image = np.array(image).transpose(1, 2, 0)
    if label_exists:
        label = np.array(label).transpose(1, 2, 0)
    # print(image.shape)
    return image, label

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality,Path_Segmentation
